In [ ]:
# Create initial Dataframe

import os
# import sys
import pandas as pd
import librosa
import soundfile as sf
from tqdm import tqdm
import numpy as np

# np.set_printoptions(threshold=10000)

#Input directory example
#/Users/michael/Desktop/test_demo

folder_path = input("Input audio path")

#Prepares arrays for dataframe
file_name = []
folder = []
audio = []
sample_rate = []
bit_depth = []
channels = []
audio_length = []

#Create dataframe
dataset = pd.DataFrame(columns = ['file_name', 'folder', 'audio', 'sample_rate', 'bit_depth', 'channels', 'audio_length', 'class_id', 'class'])
# dataset = dataset.astype({'sample_rate': int16})

# gets number of all files in a folder
total = sum([len(files) for r, d, files in os.walk(folder_path)])
pbar = tqdm(total = total)

# Iterates all the wav files in folders and subfolders
# Adds this information to dataframe.
for root, dirs, files in os.walk(folder_path, topdown=False):
    for name in files:
        if ".wav" in name:
            #variables
            base_folder_name = os.path.normpath(os.path.basename(root))
            audio_data, sr = librosa.load(root + "/" + name)
            sr_data = np.array([sr], dtype=np.int32) #places samplerate into int32 numpy array
            ob = sf.SoundFile(root + "/" + name)

            #upload data into each array
            file_name.append(name) #name of wav file
            folder.append(base_folder_name) #folder location
            audio.append(audio_data) #raw audio data. Librosa always returns 32 bit float (print(audio_data.dtype))
            sample_rate.append(sr_data) #sample rate
            bit_depth.append(ob.subtype) #returns pcm_16 which means 16bit depth
            channels.append(len(audio_data.shape)) #number of channels
            audio_length.append(audio_data.shape[0]/sr) #length of audio

            pbar.update(1)
        #skips any files thats not a wave
        else:
            pbar.update(1)
            # print(name + " is not a wav file")
            
    
pbar.close()


#Place arrays into dataframe
dataset.file_name = file_name
dataset.folder = folder
dataset.audio = audio
dataset.sample_rate = sample_rate
dataset.bit_depth = bit_depth
dataset.channels = channels
dataset.audio_length = audio_length

# print(dataset.dtypes) # does not print numpy array type

#Export dataframe
dataset.to_csv (folder_path + '/' + "test.csv", index = False, header=True)
